In [107]:
import numpy as np
import pandas as pd
import pickle

In [108]:
with open('test_set.pickle', 'rb') as file:
    test = pickle.load(file)
with open('train_set.pickle','rb') as file2:
    train = pickle.load(file2)
with open('val_set.pickle','rb') as file3:
    val = pickle.load(file3)

In [109]:
# lower_case:
def lower_case_dict(mycase,field_name):
    temp1 = []
    for i in mycase[field_name]:
        temp2 = []
        for j in i:
            temp2.append(j.lower())
        temp1.append(temp2)
    return temp1
train['tokens'] = lower_case_dict(train,'tokens')
val['tokens'] = lower_case_dict(val,'tokens')

In [110]:
def dict2tuple(mydict):
    temp1 = []
    for i in range(len(mydict['tokens'])):
        temp2 = []
        for j in range(len(mydict['tokens'][i])):
            temp2.append((mydict['tokens'][i][j],mydict['pos_tags'][i][j]))
        temp1.append(temp2)
    return temp1
train_tuple = dict2tuple(train)
val_tuple = dict2tuple(val)


    

In [126]:
# find_freq and word_tag_freq for val:
word_tag_freq = {}
tag_freq= {}
for i in train_tuple:
    for word, tag in i:
        word_tag_freq[(word, tag)] = word_tag_freq.get((word, tag), 0) + 1
        tag_freq[tag] = tag_freq.get(tag, 0) + 1

In [232]:
def find_best(word_tag_freq,tag_freq,transition_probabilities,tokenized_sentence):
    tagged_sentence = []
    const_value = 10 ** (-6)
    for word in tokenized_sentence:
        max_prob = 0
        max_tag = None
        for tag in tag_freq.keys():
            if len(tagged_sentence) != 0 :
                if (tagged_sentence[-1][1] , tag ) in transition_probabilities.keys() and word_tag_freq.get((word, tag), 0) != 0 :
                    prob = word_tag_freq.get((word, tag), 0) / tag_freq[tag] * transition_probabilities[(tagged_sentence[-1][1] , tag)]
                elif (tagged_sentence[-1][1] , tag ) not in transition_probabilities.keys() and word_tag_freq.get((word, tag), 0) != 0:
                    prob = word_tag_freq.get((word, tag), 0) / tag_freq[tag] * const_value
                elif (tagged_sentence[-1][1] , tag ) in transition_probabilities.keys() and word_tag_freq.get((word, tag), 0) == 0:
                    prob = const_value * transition_probabilities[(tagged_sentence[-1][1] , tag)]
                elif (tagged_sentence[-1][1] , tag ) not in transition_probabilities.keys() and  word_tag_freq.get((word, tag), 0) == 0:
                    prob = const_value * const_value
            else:
                if word_tag_freq.get((word, tag), 0) != 0:
                    prob = word_tag_freq.get((word, tag), 0) / tag_freq[tag]
                else:
                    prob = const_value / tag_freq[tag]
            if prob > max_prob:
                max_prob = prob
                max_tag = tag
        
        tagged_sentence.append((word, max_tag))
    return(tagged_sentence)

# محاسبه ماتریس احتمالات ترتیبی
def calculate_transition_probabilities(sentences):
    transitions = {}
    tag_counts = {}
    
    for sentence in sentences:
        for i in range(len(sentence) - 1):
            current_tag = sentence[i][1]
            next_tag = sentence[i+1][1]
            
            transitions[(current_tag, next_tag)] = transitions.get((current_tag, next_tag), 0) + 1
            tag_counts[current_tag] = tag_counts.get(current_tag, 0) + 1
    
    transition_probabilities = {}
    for transition, count in transitions.items():
        current_tag = transition[0]
        transition_probabilities[transition] = count / tag_counts[current_tag]
    
    return transition_probabilities


def find_equal_counts(Ltuple1, Ltuple2):
    t = 0
    if len(Ltuple1) == len(Ltuple2):
        for i in range(len(Ltuple1)):
            if Ltuple1[i][1] == Ltuple2[i][1]:
                t=t+1
    return(t)

In [162]:
# find_accuracy_f1_precision: in micro f1 = accuracy = precission in train:
t = 0
all_count = 0
transition_probabilities_train = calculate_transition_probabilities(train_tuple)
for i in range(len(train['tokens'])):
    tagged_sentence = find_best(word_tag_freq,tag_freq,transition_probabilities_train,train['tokens'][i])
    t = t + find_equal_counts(tagged_sentence,train_tuple[i])
    all_count = all_count + len(train_tuple[i])
print(t/all_count)

0.9407662553996443


In [163]:
# find_accuracy_f1_precision: in micro f1 = accuracy = precission in val:
t = 0
all_count = 0
for i in range(len(val['tokens'])):
    tagged_sentence = find_best(word_tag_freq,tag_freq,transition_probabilities_train,val['tokens'][i])
    t = t + find_equal_counts(tagged_sentence,val_tuple[i])
    all_count = all_count + len(val_tuple[i])
print(t/all_count)

0.8808887457736274


In [152]:
test

{'tokens': [['A',
   'few',
   'years',
   'ago',
   ',',
   'barter',
   'deals',
   'accounted',
   'for',
   'up',
   'to',
   '25-30',
   'percent',
   'of',
   'Russian',
   'exports',
   'because',
   '"',
   'thousands',
   '(',
   'of',
   ')',
   'trade',
   'companies',
   'which',
   'popped',
   'up',
   'preferred',
   'barter',
   'in',
   'the',
   'absence',
   'of',
   'reliable',
   'Russian',
   'banks',
   'and',
   'money',
   'transfer',
   'systems',
   '"',
   ',',
   'Volkova',
   'said',
   '.'],
  ['The',
   'Maoists',
   'oppose',
   'multi-party',
   'democracy',
   'and',
   'want',
   'to',
   'establish',
   'a',
   'communist',
   'state',
   '.'],
  ["O'Meara", ',', 'Fred', 'Couples'],
  ['The',
   'ruling',
   'Socialist',
   'Party',
   'last',
   'week',
   'accused',
   'Serbia',
   "'s",
   'opposition',
   'of',
   'stirring',
   'up',
   'social',
   'unrest',
   'and',
   'using',
   'workers',
   "'",
   'already',
   'difficult',
   'social',

In [164]:
# prediction test:
temp = []
for i in range(len(test['tokens'])):
    tagged_sentence = find_best(word_tag_freq,tag_freq,transition_probabilities_train,test['tokens'][i])
    temp.append(tagged_sentence)
print(temp)

[[('A', 25), ('few', 11), ('years', 24), ('ago', 30), (',', 6), ('barter', 17), ('deals', 24), ('accounted', 38), ('for', 15), ('up', 30), ('to', 35), ('25-30', 37), ('percent', 21), ('of', 15), ('Russian', 12), ('exports', 24), ('because', 15), ('"', 0), ('thousands', 24), ('(', 4), ('of', 15), (')', 5), ('trade', 37), ('companies', 24), ('which', 43), ('popped', 38), ('up', 33), ('preferred', 16), ('barter', 17), ('in', 15), ('the', 12), ('absence', 21), ('of', 15), ('reliable', 12), ('Russian', 21), ('banks', 24), ('and', 10), ('money', 21), ('transfer', 37), ('systems', 24), ('"', 0), (',', 6), ('Volkova', 22), ('said', 38), ('.', 7)], [('The', 25), ('Maoists', 11), ('oppose', 41), ('multi-party', 16), ('democracy', 21), ('and', 10), ('want', 41), ('to', 35), ('establish', 37), ('a', 12), ('communist', 16), ('state', 21), ('.', 7)], [("O'Meara", 25), (',', 11), ('Fred', 11), ('Couples', 11)], [('The', 25), ('ruling', 11), ('Socialist', 11), ('Party', 11), ('last', 16), ('week', 21)

In [180]:
output_list = []
for i in temp:
    temp_list = []
    for j in i:
        temp_list.append(j[1])
    output_list.append(temp_list)

In [192]:
test['pos_tags'] = output_list 

In [201]:
test['pos_tags']

[[25,
  11,
  24,
  30,
  6,
  17,
  24,
  38,
  15,
  30,
  35,
  37,
  21,
  15,
  12,
  24,
  15,
  0,
  24,
  4,
  15,
  5,
  37,
  24,
  43,
  38,
  33,
  16,
  17,
  15,
  12,
  21,
  15,
  12,
  21,
  24,
  10,
  21,
  37,
  24,
  0,
  6,
  22,
  38,
  7],
 [25, 11, 41, 16, 21, 10, 41, 35, 37, 12, 16, 21, 7],
 [25, 11, 11, 11],
 [25,
  11,
  11,
  11,
  16,
  21,
  38,
  12,
  42,
  21,
  15,
  12,
  30,
  16,
  21,
  10,
  39,
  24,
  27,
  30,
  16,
  16,
  21,
  15,
  29,
  16,
  24,
  7],
 [25, 11, 38, 12, 21, 10, 38, 28, 42, 16, 35, 37, 28, 15, 21, 7],
 [25,
  11,
  21,
  27,
  24,
  35,
  37,
  29,
  21,
  21,
  20,
  37,
  12,
  16,
  21,
  35,
  22,
  24,
  6,
  12,
  21,
  15,
  15,
  12,
  4,
  22,
  5,
  38,
  15,
  29,
  16,
  21,
  7],
 [25, 11],
 [25, 11, 11, 30, 11, 22, 15, 12, 21, 11, 7],
 [34, 42, 40, 24, 7],
 [25, 11, 11],
 [0,
  28,
  41,
  40,
  39,
  28,
  15,
  16,
  24,
  30,
  10,
  41,
  12,
  24,
  28,
  42,
  16,
  21,
  35,
  12,
  21,
  7],
 [25,
  1

In [203]:
file_name = 'first_pred.pickle'
with open(file_name, 'wb') as file:
    pickle.dump(test, file)

In [204]:
# test_saved
with open('first_pred.pickle', 'rb') as file4:
    temp_test = pickle.load(file4)

In [208]:
temp_test

{'tokens': [['A',
   'few',
   'years',
   'ago',
   ',',
   'barter',
   'deals',
   'accounted',
   'for',
   'up',
   'to',
   '25-30',
   'percent',
   'of',
   'Russian',
   'exports',
   'because',
   '"',
   'thousands',
   '(',
   'of',
   ')',
   'trade',
   'companies',
   'which',
   'popped',
   'up',
   'preferred',
   'barter',
   'in',
   'the',
   'absence',
   'of',
   'reliable',
   'Russian',
   'banks',
   'and',
   'money',
   'transfer',
   'systems',
   '"',
   ',',
   'Volkova',
   'said',
   '.'],
  ['The',
   'Maoists',
   'oppose',
   'multi-party',
   'democracy',
   'and',
   'want',
   'to',
   'establish',
   'a',
   'communist',
   'state',
   '.'],
  ["O'Meara", ',', 'Fred', 'Couples'],
  ['The',
   'ruling',
   'Socialist',
   'Party',
   'last',
   'week',
   'accused',
   'Serbia',
   "'s",
   'opposition',
   'of',
   'stirring',
   'up',
   'social',
   'unrest',
   'and',
   'using',
   'workers',
   "'",
   'already',
   'difficult',
   'social',

In [235]:

# محاسبه ماتریس احتمالات ظاهر شدن کلمه t-1 به شرط تگ t
def calculate_emission_probabilities_with_prev_word(sentences):
    emissions_with_prev_word = {}
    tag_counts = {}

    for sentence in sentences:
        for i in range(1, len(sentence)):
            current_word, current_tag = sentence[i]
            prev_word = sentence[i-1][0]

            emissions_with_prev_word[(prev_word, current_tag)] = emissions_with_prev_word.get((prev_word, current_tag), 0) + 1
            tag_counts[current_tag] = tag_counts.get(current_tag, 0) + 1

    emission_prob_with_prev_word = {}
    for emission, count in emissions_with_prev_word.items():
        current_tag = emission[1]
        emission_prob_with_prev_word[emission] = count / tag_counts[current_tag]
    return emission_prob_with_prev_word



In [233]:
# find_best_soal3 
def find_best_3(word_tag_freq,tag_freq,transition_probabilities,emission_probabilities_with_prev_word,tokenized_sentence):
    tagged_sentence = []
    const_value = 10 ** (-6)
    for word_index in range(len(tokenized_sentence)):
        max_prob = 0
        max_tag = None
        for tag in tag_freq.keys():
            if len(tagged_sentence) != 0 :
                if (tagged_sentence[-1][1] , tag ) in transition_probabilities.keys() and word_tag_freq.get((tokenized_sentence[word_index], tag), 0) != 0 and (tokenized_sentence[word_index - 1],tag) in emission_probabilities_with_prev_word.keys() :
                    prob = word_tag_freq.get((tokenized_sentence[word_index], tag), 0) / tag_freq[tag] * transition_probabilities[(tagged_sentence[-1][1] , tag)] * emission_probabilities_with_prev_word[(tokenized_sentence[word_index - 1],tag)]
                elif (tagged_sentence[-1][1] , tag ) not in transition_probabilities.keys() and word_tag_freq.get((tokenized_sentence[word_index], tag), 0) != 0 and (tokenized_sentence[word_index - 1],tag) in emission_probabilities_with_prev_word.keys() :
                    prob = word_tag_freq.get((tokenized_sentence[word_index], tag), 0) / tag_freq[tag] * const_value * emission_probabilities_with_prev_word[(tokenized_sentence[word_index - 1],tag)]
                elif (tagged_sentence[-1][1] , tag ) in transition_probabilities.keys() and word_tag_freq.get((tokenized_sentence[word_index], tag), 0) == 0 and (tokenized_sentence[word_index - 1],tag) in emission_probabilities_with_prev_word.keys():
                    prob = const_value * transition_probabilities[(tagged_sentence[-1][1] , tag)] * emission_probabilities_with_prev_word[(tokenized_sentence[word_index - 1],tag)]
                elif (tagged_sentence[-1][1] , tag ) not in transition_probabilities.keys() and  word_tag_freq.get((tokenized_sentence[word_index], tag), 0) == 0 and (tokenized_sentence[word_index - 1],tag) in emission_probabilities_with_prev_word.keys():
                    prob = const_value * const_value * emission_probabilities_with_prev_word[(tokenized_sentence[word_index - 1],tag)]
                
                elif (tagged_sentence[-1][1] , tag ) in transition_probabilities.keys() and word_tag_freq.get((tokenized_sentence[word_index], tag), 0) != 0 and (tokenized_sentence[word_index - 1],tag) not in emission_probabilities_with_prev_word.keys() :
                    prob = word_tag_freq.get((tokenized_sentence[word_index], tag), 0) / tag_freq[tag] * transition_probabilities[(tagged_sentence[-1][1] , tag)] * const_value 
                elif (tagged_sentence[-1][1] , tag ) not in transition_probabilities.keys() and word_tag_freq.get((tokenized_sentence[word_index], tag), 0) != 0 and (tokenized_sentence[word_index - 1],tag) not in emission_probabilities_with_prev_word.keys() :
                    prob = word_tag_freq.get((tokenized_sentence[word_index], tag), 0) / tag_freq[tag] * const_value * const_value
                elif (tagged_sentence[-1][1] , tag ) in transition_probabilities.keys() and word_tag_freq.get((tokenized_sentence[word_index], tag), 0) == 0 and (tokenized_sentence[word_index - 1],tag) not in emission_probabilities_with_prev_word.keys():
                    prob = const_value * transition_probabilities[(tagged_sentence[-1][1] , tag)] * const_value
                elif (tagged_sentence[-1][1] , tag ) not in transition_probabilities.keys() and  word_tag_freq.get((tokenized_sentence[word_index], tag), 0) == 0 and (tokenized_sentence[word_index - 1],tag) not in emission_probabilities_with_prev_word.keys():
                    prob = const_value * const_value * const_value
            else:
                if word_tag_freq.get((tokenized_sentence[word_index], tag), 0) != 0:
                    prob = word_tag_freq.get((tokenized_sentence[word_index], tag), 0) / tag_freq[tag]
                else:
                    prob = const_value / tag_freq[tag]
            if prob > max_prob:
                max_prob = prob
                max_tag = tag
        
        tagged_sentence.append((tokenized_sentence[word_index], max_tag))
    return(tagged_sentence)

In [236]:
# find_accuracy_f1_precision: in micro f1 = accuracy = precission in train:
t = 0
all_count = 0
transition_probabilities_train = calculate_transition_probabilities(train_tuple)
emission_probabilities_with_prev_word_train = calculate_emission_probabilities_with_prev_word(train_tuple)
for i in range(len(train['tokens'])):
    tagged_sentence = find_best_3(word_tag_freq,tag_freq,transition_probabilities_train,emission_probabilities_with_prev_word_train,train['tokens'][i])
    t = t + find_equal_counts(tagged_sentence,train_tuple[i])
    all_count = all_count + len(train_tuple[i])
print(t/all_count)

0.943956633445326


In [237]:
# find_accuracy_f1_precision: in micro f1 = accuracy = precission in val:
t = 0
all_count = 0
for i in range(len(val['tokens'])):
    tagged_sentence = find_best_3(word_tag_freq,tag_freq,transition_probabilities_train,emission_probabilities_with_prev_word_train,val['tokens'][i])
    t = t + find_equal_counts(tagged_sentence,val_tuple[i])
    all_count = all_count + len(val_tuple[i])
print(t/all_count)

0.8652712928674932


In [238]:
# prediction test:
temp = []
for i in range(len(test['tokens'])):
    tagged_sentence = find_best_3(word_tag_freq,tag_freq,transition_probabilities_train,emission_probabilities_with_prev_word_train,test['tokens'][i])
    temp.append(tagged_sentence)
print(temp)

[[('A', 25), ('few', 11), ('years', 24), ('ago', 30), (',', 6), ('barter', 0), ('deals', 24), ('accounted', 38), ('for', 15), ('up', 30), ('to', 35), ('25-30', 37), ('percent', 21), ('of', 15), ('Russian', 12), ('exports', 24), ('because', 15), ('"', 0), ('thousands', 24), ('(', 4), ('of', 15), (')', 12), ('trade', 21), ('companies', 24), ('which', 43), ('popped', 42), ('up', 30), ('preferred', 16), ('barter', 24), ('in', 15), ('the', 12), ('absence', 21), ('of', 15), ('reliable', 12), ('Russian', 21), ('banks', 24), ('and', 10), ('money', 21), ('transfer', 15), ('systems', 12), ('"', 0), (',', 6), ('Volkova', 0), ('said', 38), ('.', 7)], [('The', 25), ('Maoists', 11), ('oppose', 41), ('multi-party', 12), ('democracy', 21), ('and', 10), ('want', 41), ('to', 35), ('establish', 37), ('a', 12), ('communist', 16), ('state', 21), ('.', 7)], [("O'Meara", 25), (',', 11), ('Fred', 11), ('Couples', 11)], [('The', 25), ('ruling', 11), ('Socialist', 24), ('Party', 15), ('last', 16), ('week', 21),

In [239]:
output_list = []
for i in temp:
    temp_list = []
    for j in i:
        temp_list.append(j[1])
    output_list.append(temp_list)

In [240]:
test['pos_tags'] = output_list 

In [241]:
test['pos_tags']

[[25,
  11,
  24,
  30,
  6,
  0,
  24,
  38,
  15,
  30,
  35,
  37,
  21,
  15,
  12,
  24,
  15,
  0,
  24,
  4,
  15,
  12,
  21,
  24,
  43,
  42,
  30,
  16,
  24,
  15,
  12,
  21,
  15,
  12,
  21,
  24,
  10,
  21,
  15,
  12,
  0,
  6,
  0,
  38,
  7],
 [25, 11, 41, 12, 21, 10, 41, 35, 37, 12, 16, 21, 7],
 [25, 11, 11, 11],
 [25,
  11,
  24,
  15,
  16,
  21,
  38,
  15,
  27,
  21,
  15,
  12,
  30,
  16,
  21,
  10,
  39,
  24,
  27,
  21,
  16,
  16,
  21,
  15,
  29,
  16,
  24,
  7],
 [25, 11, 40, 12, 21, 10, 38, 28, 42, 16, 35, 37, 28, 15, 21, 7],
 [25,
  11,
  21,
  27,
  24,
  35,
  37,
  29,
  21,
  21,
  20,
  37,
  12,
  16,
  21,
  35,
  22,
  24,
  6,
  12,
  21,
  15,
  15,
  12,
  4,
  22,
  5,
  38,
  15,
  29,
  16,
  21,
  7],
 [25, 11],
 [25, 11, 11, 30, 11, 22, 15, 12, 21, 11, 7],
 [34, 42, 40, 24, 7],
 [25, 11, 11],
 [0,
  28,
  41,
  40,
  39,
  28,
  15,
  16,
  24,
  30,
  10,
  37,
  12,
  24,
  28,
  42,
  22,
  21,
  35,
  12,
  21,
  7],
 [25,
  11

In [242]:
file_name = 'second_pred.pickle'
with open(file_name, 'wb') as file:
    pickle.dump(test, file)

In [243]:
# test_saved
with open('second_pred.pickle', 'rb') as file5:
    temp_test = pickle.load(file5)

In [245]:
temp_test['pos_tags']

[[25,
  11,
  24,
  30,
  6,
  0,
  24,
  38,
  15,
  30,
  35,
  37,
  21,
  15,
  12,
  24,
  15,
  0,
  24,
  4,
  15,
  12,
  21,
  24,
  43,
  42,
  30,
  16,
  24,
  15,
  12,
  21,
  15,
  12,
  21,
  24,
  10,
  21,
  15,
  12,
  0,
  6,
  0,
  38,
  7],
 [25, 11, 41, 12, 21, 10, 41, 35, 37, 12, 16, 21, 7],
 [25, 11, 11, 11],
 [25,
  11,
  24,
  15,
  16,
  21,
  38,
  15,
  27,
  21,
  15,
  12,
  30,
  16,
  21,
  10,
  39,
  24,
  27,
  21,
  16,
  16,
  21,
  15,
  29,
  16,
  24,
  7],
 [25, 11, 40, 12, 21, 10, 38, 28, 42, 16, 35, 37, 28, 15, 21, 7],
 [25,
  11,
  21,
  27,
  24,
  35,
  37,
  29,
  21,
  21,
  20,
  37,
  12,
  16,
  21,
  35,
  22,
  24,
  6,
  12,
  21,
  15,
  15,
  12,
  4,
  22,
  5,
  38,
  15,
  29,
  16,
  21,
  7],
 [25, 11],
 [25, 11, 11, 30, 11, 22, 15, 12, 21, 11, 7],
 [34, 42, 40, 24, 7],
 [25, 11, 11],
 [0,
  28,
  41,
  40,
  39,
  28,
  15,
  16,
  24,
  30,
  10,
  37,
  12,
  24,
  28,
  42,
  22,
  21,
  35,
  12,
  21,
  7],
 [25,
  11